In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

DATA_DIR = '/content/drive/MyDrive/sichuan_data' 
GDP_RASTER = f'{DATA_DIR}/china_gdp_1km_2020.tif'
SICHUAN_SHP = f'{DATA_DIR}/sichuan_province.shp'

print(GDP_RASTER)
print(SICHUAN_SHP)

In [ ]:
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import Point
import ee

In [ ]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [ ]:
sichuan = gpd.read_file(SICHUAN_SHP)
sichuan = sichuan.to_crs(epsg=4326)
sichuan_geom = [sichuan.unary_union.__geo_interface__]

sichuan.head()

In [ ]:
with rasterio.open(GDP_RASTER) as src:
    out_image, out_transform = mask(src, sichuan_geom, crop=True)
    out_meta = src.meta.copy()

gdp_data = out_image[0]
gdp_data[gdp_data <= 0] = np.nan  

rows, cols = gdp_data.shape
print('GDP raster (Sichuan) shape:', gdp_data.shape)

In [ ]:
def rowcol_to_lonlat(row, col, transform):
    x, y = rasterio.transform.xy(transform, row, col)
    return float(x), float(y)


N = 4000  
rng = np.random.default_rng(42)

indices = []
values = []

while len(indices) < N:
    r = rng.integers(0, rows)
    c = rng.integers(0, cols)
    val = gdp_data[r, c]
    if np.isnan(val):
        continue
    indices.append((r, c))
    values.append(val)

values = np.array(values)
lonlat = [rowcol_to_lonlat(r, c, out_transform) for (r, c) in indices]
lons = np.array([p[0] for p in lonlat])
lats = np.array([p[1] for p in lonlat])

print('Number of samples:', len(values))

In [ ]:
viirs_col = ee.ImageCollection('NOAA/VIIRS/DNB/ANNUAL_V22') \    .filterDate('2020-01-01', '2021-01-01')
viirs = viirs_col.median().select('avg_rad')

In [ ]:
features = []
for lon, lat, gdp_val in zip(lons, lats, values):
    geom = ee.Geometry.Point([lon, lat])
    feat = ee.Feature(geom, {'gdp': float(gdp_val)})
    features.append(feat)

fc = ee.FeatureCollection(features)

In [ ]:
sampled = viirs.sampleRegions(
    collection=fc,
    scale=1000,
    geometries=True
)

sample_dict = sampled.getInfo()

ntl_list = []
gdp_list = []
lon_list = []
lat_list = []

for f in sample_dict['features']:
    props = f['properties']
    geom = f['geometry']['coordinates']
    if 'avg_rad' not in props:
        continue
    lon_list.append(float(geom[0]))
    lat_list.append(float(geom[1]))
    gdp_list.append(float(props['gdp']))
    ntl_list.append(float(props['avg_rad']))

print('Final sample size:', len(gdp_list))

In [ ]:
import pandas as pd
import os

out_dir = os.path.join(DATA_DIR, 'processed')
os.makedirs(out_dir, exist_ok=True)

df = pd.DataFrame({
    'lon': lon_list,
    'lat': lat_list,
    'gdp': gdp_list,
    'ntl': ntl_list,
})

csv_path = os.path.join(out_dir, 'sample_points.csv')
df.to_csv(csv_path, index=False)
csv_path